# Reduction of spin flip data from McStas simulation

This notebook contains an example of polarized reflectometry data reduction.
The dataset comes from a McStas simulation of the ESTIA instrument.


Four samples were simulated:

* `supermirror` - a perfect nonmagnetic supermirror
* `magnetic_supermirror` - a magnetic supermirror with high reflectivity for one spin state and low reflectivity for the other
* `magnetic_supermirror_2` - a magnetic supermirror with slightly different reflectivity curve than `magnetic_supermirror`
* `spin_flip_sample` - a magnetic supermirror that also causes some incident neutrons to spin flip with 10% probability

Each sample was measured for four different flipper settings `offoff`, `offon`, `onoff`, `onon`, corresponding to the polarizer and analyzer flipper setting.

In the data reduction procedure  `supermirror` and `magnetic_supermirror_2` will be used to calibrate the parameters of the instrument.

We start by importing packages and creating the workflow:

In [ ]:
%matplotlib widget
import scipp as sc
import plopp as pp

from ess.reflectometry.types import *
from ess.reflectometry.supermirror import CriticalEdge

from ess.estia.types import *
from ess.estia import EstiaWorkflow
from ess.estia.load import load_mcstas_events
from ess.estia.data import (
    estia_mcstas_spin_flip_example,
    estia_mcstas_spin_flip_example_groundtruth,
    estia_mcstas_spin_flip_example_download_all_to_cache
)
from ess.estia.calibration import (
    reflectivity_provider,
    reflectivity_provider_calibrate_on_lz
)

wf = EstiaWorkflow()
wf.insert(load_mcstas_events)
wf.insert(reflectivity_provider)

wf[YIndexLimits]  = sc.scalar(35), sc.scalar(64)
wf[ZIndexLimits] = sc.scalar(0), sc.scalar(14 * 32)
wf[BeamDivergenceLimits] = sc.scalar(-0.75, unit='deg'), sc.scalar(0.75, unit='deg')
wf[WavelengthBins] = sc.geomspace('wavelength', 3.5, 12, 301, unit='angstrom')
wf[QBins] = sc.linspace('Q', 0.001, 0.1, 200, unit='1/angstrom')

# Reference sample is perfect supermirror with reflectivity = 1 everywhere
wf[CriticalEdge] = sc.scalar(float('inf'), unit='1/angstrom')

# There is no proton current data in the McStas files, here we just add some fake proton current
# data to make the workflow run.
wf[ProtonCurrent[SampleRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})
wf[ProtonCurrent[ReferenceRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})

Download the data: (might take ~2 minutes depending on your internet connection)

In [ ]:
%%time
estia_mcstas_spin_flip_example_download_all_to_cache()

## Reducing the data

First each dataset is loaded and reduced separately.
The datasets are reduced "as references" or "as samples" depending on how they are supposed to be used.
`supermirror` and `magnetic_supermirror_2` are reduced as references, and `supermirror` and `magnetic_supermirror_2` are reduced as samples.

In [ ]:
references = {}
for sample in (
    'supermirror',
    'magnetic_supermirror_2',
):
    references[sample] = []
    for flipper_setting in ('offoff', 'offon', 'onoff', 'onon'):
        w = wf.copy()
        w[RawDetectorData[SampleRun]] = sc.io.load_hdf5(estia_mcstas_spin_flip_example('spin_flip_sample', 'offoff'))
        w[RawDetectorData[ReferenceRun]] = sc.io.load_hdf5(estia_mcstas_spin_flip_example(sample, flipper_setting))
        references[sample].append(w.compute(Reference))

        # We need to unalign all coords of the references to use
        # them in the calibration procedure
        for c in references[sample][-1].coords:
            references[sample][-1].coords.set_aligned(c, False)


samples = {}
for sample in (
    'spin_flip_sample',
    'magnetic_supermirror'
):
    samples[sample] = []
    for flipper_setting in ('offoff', 'offon', 'onoff', 'onon'):
        w = wf.copy()
        w[RawDetectorData[SampleRun]] = sc.io.load_hdf5(estia_mcstas_spin_flip_example(sample, flipper_setting))
        w[RawDetectorData[ReferenceRun]] = sc.io.load_hdf5(estia_mcstas_spin_flip_example('supermirror', 'offoff'))
        samples[sample].append(w.compute(Sample))

Here we load the ground truth reflectivity curves for the `up` respectively `down` spin state.

In [ ]:
Rdown = sc.io.load_hdf5(estia_mcstas_spin_flip_example_groundtruth('down'))
Rup = sc.io.load_hdf5(estia_mcstas_spin_flip_example_groundtruth('up'))

The calibration is performed automatically in the workflow and the user does not have to care about the calibration parameters.
But for debugging and troubleshooting it is good to be able to visualize the calibration parameters obtained from the reference measurements.
To do this we first calculate the calibration parameters from our nonmagnetic and magnetic reference:

In [ ]:
from ess.estia.calibration import PolarizationCalibrationParameters
calibration = PolarizationCalibrationParameters.from_reference_measurements(
    references['supermirror'],
    references['magnetic_supermirror_2']
)

From the `PolarizationCalibrationParameters` object we can obtain the individual calibration parameters, such as the effective polarization of the polarizer when the polarizer flipper is off $Pp$ and the reference intensity $I_0$.

In [ ]:
calibration.Pp

In [ ]:
from ess.reflectometry.figures import wavelength_theta_figure

wavelength_theta_figure(calibration.Ap['blade', 5:9], q_edges_to_display=[sc.scalar(0.016, unit='1/angstrom')], vmin=0.1, vmax=3)

Above we see the calibration parameter $A_p$, the effective polarization of the analyzer when the analyzer flipper is off.
The effective polarization should be close to constant but weakly wavelength dependent if the analyzer is working well.
We can see that the analyzer appears to be working well in the region where $Q>0.016$ angstrom (line in figure illustrates the cutoff).
Below that $Q$ limit the magnetic supermirror reflects both spin states, so there we can't distinguish between them and the calibration fails.

Note that the calibration result is quite noisy. Depending on the amount of counts and the number of bins used that can be the case, especially when the calibration is performed on the 2D grid as above.
If we expect that our polarizing components are good, and we are limited by low statistics (the common case) it is sufficient to do the calibration directly on the Q-grid to increase the counts per bin and reduce the noise.
This is controlled by using either the `ess.estia.calibration.reflectivity_provider` or the `ess.estia.calibration.reflectivity_provider_calibrate_on_lz` provider in the workflow.
The recommended default is to use the `ess.estia.calibration.reflectivity_provider` that does the calibration directly on the $Q$ grid.

# Computing reflectivity with polarization correction

## Spin flip sample: magnetic supermirror with 10% spin flip probability

In [ ]:
w = wf.copy()
w.insert(reflectivity_provider)

(
    w[Intensity[NonMagneticReference, OffOff]],
    w[Intensity[NonMagneticReference, OffOn]],
    w[Intensity[NonMagneticReference, OnOff]],
    w[Intensity[NonMagneticReference, OnOn]]
) = references['supermirror']
(
    w[Intensity[MagneticReference, OffOff]],
    w[Intensity[MagneticReference, OffOn]],
    w[Intensity[MagneticReference, OnOff]],
    w[Intensity[MagneticReference, OnOn]]
) = references['magnetic_supermirror_2']
(
    w[Intensity[MagneticSample, OffOff]],
    w[Intensity[MagneticSample, OffOn]],
    w[Intensity[MagneticSample, OnOff]],
    w[Intensity[MagneticSample, OnOn]]
) = samples['spin_flip_sample']

R = w.compute(PolarizedReflectivityOverQ)

labels = ['$R_{\\downarrow\\downarrow}$', '$R_{\\downarrow\\uparrow}$', '$R_{\\uparrow\\downarrow}$', '$R_{\\uparrow\\uparrow}$']
p = pp.plot(
    {
        f'{labels[i]}': r for i, r in enumerate(R)
    } | {'True $R_{\\downarrow\\downarrow}$': Rdown * 9/10, 'True $R_{\\uparrow\\uparrow}$': Rup * 9/10},
    title='Reflectivities, corrected, spin_flip_sample, calibrate on $Q$-grid',
    norm='log',
    vmin=2e-5,
    vmax=1.5
)
p

To illustrate the difference, here are the same results if we calibrate on the 2D grid instead of on the $Q$-grid, the results are similar but a bit more noisy:

In [ ]:
w = wf.copy()
w.insert(reflectivity_provider_calibrate_on_lz)

(
    w[Intensity[NonMagneticReference, OffOff]],
    w[Intensity[NonMagneticReference, OffOn]],
    w[Intensity[NonMagneticReference, OnOff]],
    w[Intensity[NonMagneticReference, OnOn]]
) = references['supermirror']
(
    w[Intensity[MagneticReference, OffOff]],
    w[Intensity[MagneticReference, OffOn]],
    w[Intensity[MagneticReference, OnOff]],
    w[Intensity[MagneticReference, OnOn]]
) = references['magnetic_supermirror_2']
(
    w[Intensity[MagneticSample, OffOff]],
    w[Intensity[MagneticSample, OffOn]],
    w[Intensity[MagneticSample, OnOff]],
    w[Intensity[MagneticSample, OnOn]]
) = samples['spin_flip_sample']

R = w.compute(PolarizedReflectivityOverQ)

labels = ['$R_{\\downarrow\\downarrow}$', '$R_{\\downarrow\\uparrow}$', '$R_{\\uparrow\\downarrow}$', '$R_{\\uparrow\\uparrow}$']
p = pp.plot(
    {
        f'{labels[i]}': r for i, r in enumerate(R)
    } | {'True $R_{\\downarrow\\downarrow}$': Rdown * 9/10, 'True $R_{\\uparrow\\uparrow}$': Rup * 9/10},
    title='Reflectivities, corrected, spin_flip_sample,  calibrate on $LZ$-grid',
    norm='log',
    vmin=2e-5,
    vmax=1.5
)
p

## Magnetic supermirror sample - 0% spin flip probability

In [ ]:
w = wf.copy()
w.insert(reflectivity_provider)

(
    w[Intensity[NonMagneticReference, OffOff]],
    w[Intensity[NonMagneticReference, OffOn]],
    w[Intensity[NonMagneticReference, OnOff]],
    w[Intensity[NonMagneticReference, OnOn]]
) = references['supermirror']
(
    w[Intensity[MagneticReference, OffOff]],
    w[Intensity[MagneticReference, OffOn]],
    w[Intensity[MagneticReference, OnOff]],
    w[Intensity[MagneticReference, OnOn]]
) = references['magnetic_supermirror_2']
(
    w[Intensity[MagneticSample, OffOff]],
    w[Intensity[MagneticSample, OffOn]],
    w[Intensity[MagneticSample, OnOff]],
    w[Intensity[MagneticSample, OnOn]]
) = samples['magnetic_supermirror']


R = w.compute(PolarizedReflectivityOverQ)

labels = ['$R_{\\downarrow\\downarrow}$', '$R_{\\downarrow\\uparrow}$', '$R_{\\uparrow\\downarrow}$', '$R_{\\uparrow\\uparrow}$']
p = pp.plot(
    {
        f'{labels[i]}': r for i, r in enumerate(R)
    } | {'True $R_{\\downarrow\\downarrow}$': Rdown, 'True $R_{\\uparrow\\uparrow}$': Rup},
    title='Reflectivities, corrected, magnetic_supermirror',
    norm='log',
    vmin=2e-5,
    vmax=1.5
)
p

Note that we do not get zero reflectivity in the mixed spin channels as we should get if the reduction was perfect. That is because the calibration is done on reference measurements with noise. The more counts we have in the reference measurements the better results we can expect.

And again, to illustrate the difference, here are the same results if we calibrate on the 2D grid instead of on the $Q$-grid, the results are similar but a bit more noisy:

In [ ]:
w = wf.copy()
w.insert(reflectivity_provider_calibrate_on_lz)

(
    w[Intensity[NonMagneticReference, OffOff]],
    w[Intensity[NonMagneticReference, OffOn]],
    w[Intensity[NonMagneticReference, OnOff]],
    w[Intensity[NonMagneticReference, OnOn]]
) = references['supermirror']
(
    w[Intensity[MagneticReference, OffOff]],
    w[Intensity[MagneticReference, OffOn]],
    w[Intensity[MagneticReference, OnOff]],
    w[Intensity[MagneticReference, OnOn]]
) = references['magnetic_supermirror_2']

(
    w[Intensity[MagneticSample, OffOff]],
    w[Intensity[MagneticSample, OffOn]],
    w[Intensity[MagneticSample, OnOff]],
    w[Intensity[MagneticSample, OnOn]]
) = samples['magnetic_supermirror']

R = w.compute(PolarizedReflectivityOverQ)

labels = ['$R_{\\downarrow\\downarrow}$', '$R_{\\downarrow\\uparrow}$', '$R_{\\uparrow\\downarrow}$', '$R_{\\uparrow\\uparrow}$']
p = pp.plot(
    {
        f'{labels[i]}': r for i, r in enumerate(R)
    } | {'True $R_{\\downarrow\\downarrow}$': Rdown, 'True $R_{\\uparrow\\uparrow}$': Rup},
    title='Reflectivities, corrected, magnetic_supermirror, calibrate on $LZ$-grid',
    norm='log',
    vmin=2e-5,
    vmax=1.5
)
p